# Use Vault and Secret REST API to store secret in vault and utilize it inside Python function with `ibm-watsonx-ai`

This notebook demonstrates an example for using Vault and Secret REST API services to store secrets in order to utilize it inside Python function. It contains steps and code to work with [ibm-watsonx-ai](https://pypi.python.org/pypi/ibm-watsonx-ai) library available in PyPI repository.

Some familiarity with Python is helpful. This notebook uses Python 3.12.

## Learning goals

The learning goals of this notebook are:

-  Store secret in Vault using Secret REST API.
-  Utilize secret inside Python function.

## Contents

This notebook contains the following parts:

1. [Setup](#setup)
2. [Secret definition](#define-secret)
3. [Python function definition](#define-function)
4. [Testing Python function locally](#testing-locally)
5. [Deployment of Python function](#testing-locally)
6. [Testing deployed function](#testing-deployment)
7. [Clean up](#cleanup)
8. [Summary and next steps](#summary)

<a id="setup"></a>
## Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Contact with your Cloud Pak for Data administrator and ask them for your account credentials

### Install `ibm-watsonx-ai`
**Note:** `ibm-watsonx-ai` documentation can be found <a href="https://ibm.github.io/watsonx-ai-python-sdk/index.html" target="_blank" rel="noopener no referrer">here</a>.

In [1]:
%pip install -U "ibm-watsonx-ai" | tail -n 1

#### Define credentials

Authenticate the watsonx.ai Runtime on IBM Cloud Pak for Data. You need to provide the **admin's** `username` and the platform `url`.

In [2]:
import getpass

url = "PASTE THE PLATFORM URL HERE"
username = "PASTE YOUR USERNAME HERE"
password = getpass.getpass("Enter your watsonx.ai password and hit enter: ")

Enter your watsonx.ai password and hit enter:  ········


Use the **admin's** password to authenticate watsonx.ai Runtime:

In [3]:
from ibm_watsonx_ai import Credentials

credentials = Credentials(
    username=username,
    password=password,
    url=url,
    instance_id="openshift",
    version="5.2",
)

#### Create `APIClient` instance

In [4]:
from ibm_watsonx_ai import APIClient

client = APIClient(credentials)

### Working with spaces

First of all, you need to create a space that will be used for your work. If you do not have space already created, you can use `{PLATFORM_URL}/ml-runtime/spaces?context=icp4data` to create one.

- Click New Deployment Space
- Create an empty space
- Go to space `Settings` tab
- Copy `space_id` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watsonx-ai-samples/blob/master/cpd5.2/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign space ID below

In [5]:
space_id = "PASTE YOUR SPACE ID HERE"

You can use `list` method to print all existing spaces.

In [ ]:
client.spaces.list(limit=10)

To be able to interact with all resources available in watsonx.ai Runtime, you need to set **space** which you will be using.

In [6]:
client.set.default_space(space_id)

'SUCCESS'

# Define the secret
<a id="define-secret"></a>

Create a secret utilizing the credentials defined above.

**Note**: Vault and Secret REST API documentation is available [here](https://www.ibm.com/docs/en/software-hub/5.2.x?topic=2-managing-secrets-vaults).

### Fetch Vault URN

In order to use the Secret REST API, you need to retrieve the Vault URN, which points to the vault where your secret will be stored.

#### Define the Cloud Pak for Data host

In [7]:
import os

try:
    host = os.environ.get("RUNTIME_ENV_APSX_URL")
except KeyError:
    host = input("Please enter your Cloud Pak for Data URL (hit enter): ")

#### Define request parameters

In [8]:
provider_name = "internal"

url = f"{host}/zen-data/v2/vaults"
headers = client.get_headers()
params = {"provider_name": provider_name}

#### List available vaults

In [9]:
import json
import requests

response = requests.get(url, headers=headers, params=params)
vault_list = response.json()["vaults"]

print(json.dumps(vault_list, indent=2))

[
  {
    "created_at": "2025-05-03T14:04:30.970485Z",
    "created_by": "admin",
    "invalid_auth": false,
    "invalid_owner": false,
    "owned_by": "admin",
    "owner_uid": "1000330999",
    "provider_name": "Internal vault",
    "requires_auth_update": false,
    "secret_preview": "secret-name",
    "total_secrets": 1,
    "updated_at": "2025-05-03T14:04:30.970485Z",
    "vault_name": "Internal vault",
    "vault_type": "internal",
    "vault_urn": "0000000000:internal"
  }
]


#### Specify Vault URN

In [10]:
vault_urn = vault_list[0]["vault_urn"]

vault_urn

'0000000000:internal'

### Store secret in Vault

In order to utilize your secret inside the Python function, it needs to be stored in the Vault.

#### Define request parameters

In [11]:
secret_name = "secret-name"

url = f"{host}/zen-data/v2/secrets"
headers = client._get_headers(zen=True)
payload = {
    "secret_name": secret_name,
    "description": "This secret was created for a demonstration of Vault and Secret REST API",
    "secret": {"credentials": {"username": username, "password": password}},
    "type": "credentials",
    "vault_urn": vault_urn,
}

#### Store the secret

In [12]:
response = requests.post(url, headers=headers, json=payload)

print(json.dumps(response.json(), indent=2))

{
  "errors": [
    {
      "code": "service_instance_exist",
      "target": {
        "name": "field",
        "type": "display_name"
      }
    }
  ],
  "status_code": 409,
  "trace": "08eb472a-805f-4cb5-ba31-0a812776fae3"
}


# Define the Python function
<a id="define-function"></a>

Create a Python function which utilizes the secret defined above.

In [13]:
def deployable_function_with_secret(space_id=space_id, secret_name=secret_name):
    import os
    import requests

    from ibm_watsonx_ai import APIClient, Credentials

    host = os.environ.get("RUNTIME_ENV_APSX_URL")
    credentials = Credentials(
        instance_id="openshift",
        url=host,
        version="5.2",
    )
    client = APIClient(credentials)

    def get_secret_urn(host, secret_name):
        url = f"{host}/zen-data/v2/secrets"
        headers = client._get_headers(zen=True)
        params = {"secret_name": secret_name}

        response = requests.get(url, headers=headers, params=params)
        secret_list = response.json()["secrets"]

        return secret_list[0]["secret_urn"]

    def get_secret_details(host, secret_urn):
        url = f"{host}/zen-data/v2/secrets/{secret_urn}"
        headers = client._get_headers(zen=True)

        response = requests.get(url, headers=headers, verify=False)
        secret_details = response.json()

        return secret_details["data"]["secret"]["credentials"]

    secret_urn = get_secret_urn(host, secret_name)
    client_credentials = get_secret_details(host, secret_urn)

    vault_username = client_credentials["username"]
    vault_password = client_credentials["password"]

    vault_credentials = Credentials(
        url=os.environ.get("RUNTIME_ENV_APSX_URL"),
        username=vault_username,
        password=vault_password,
        instance_id="openshift",
        version="5.2",
    )

    client_from_vault = APIClient(credentials=vault_credentials, space_id=space_id)

    def score(payload):
        stored_models = client_from_vault.repository.list()
        use_score_token_values = stored_models["ID"].tolist()

        score_response = {
            "predictions": [
                {"fields": ["use_score_token"], "values": [use_score_token_values]}
            ]
        }

        return score_response

    return score


score = deployable_function_with_secret()

<a id="testing-locally"></a>
## Testing function locally

You can test your Python function locally before deploying it to the space.

In [14]:
local_function = deployable_function_with_secret(space_id, secret_name)

payload = {"input_data": [{"values": [1]}]}
result = local_function(payload)

print(json.dumps(result, indent=2))

{
  "predictions": [
    {
      "fields": [
        "use_score_token"
      ],
      "values": [
        []
      ]
    }
  ]
}


## Store and deploy the function

Before you can deploy the function, you must store it in your watsonx.ai Runtime.

### Store the function
In order to deploy the function, you need to specify your software specification.

#### Get software specification ID

In [15]:
software_spec_id = client.software_specifications.get_id_by_name("runtime-25.1-py3.12")

#### Store the function in watsonx.ai Runtime

In [16]:
store_function_meta_props = {
    client.repository.FunctionMetaNames.NAME: "Deployable function with secret from Vault",
    client.repository.FunctionMetaNames.SOFTWARE_SPEC_ID: software_spec_id,
}

function_artifact = client.repository.store_function(
    function=deployable_function_with_secret,
    meta_props=store_function_meta_props,
)

#### Get ID of deployed function

In [17]:
function_id = client.repository.get_function_id(function_artifact)

function_id

'ffc713f0-50c5-45be-ae95-afba9f5566a2'

### Deploy the function

After storing the function in your watsonx.ai Runtime, you can deploy it to your deployment space.

In [18]:
deployment_meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: "Deployed function with secret from Vault",
    client.deployments.ConfigurationMetaNames.ONLINE: {},
}

deployment_details = client.deployments.create(
    artifact_id=function_id,
    meta_props=deployment_meta_props,
)



######################################################################################

Synchronous deployment creation for id: 'ffc713f0-50c5-45be-ae95-afba9f5566a2' started

######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
........
ready


-----------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_id='db0ebd69-bbd6-4f7d-9a06-503d0e889ffa'
-----------------------------------------------------------------------------------------------




<a id="testing-deployment"></a>
## Test the deployed function

You can use `ibm-watsonx-ai` Python SDK or REST API to send data to your function deployment for processing in exactly the same way you send data to model deployments for processing.

#### Test using `ibm-watsonx-ai` Python SDK

In [19]:
payload = {"input_data": [{"values": [1]}]}

deployment_id = client.deployments.get_id(deployment_details)
result = client.deployments.score(deployment_id, payload)

if "error" in result:
    print(result["error"])
else:
    print(json.dumps(result, indent=2))

{
  "predictions": [
    {
      "fields": [
        "use_score_token"
      ],
      "values": [
        [
          "ffc713f0-50c5-45be-ae95-afba9f5566a2"
        ]
      ]
    }
  ]
}


#### Get deployment endpoint URL for REST API testing

In [ ]:
deployment_endpoint_url = client.deployments.get_scoring_href(deployment_details)

deployment_endpoint_url

<a id="cleanup"></a>
## Cleanup

If you want to clean up all created assets:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

please follow up this sample [notebook](https://github.com/IBM/watsonx-ai-samples/blob/master/cpd5.2/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>
## Summary and next steps
In this notebook, you created a Python function that receives HTML canvas image data and then processes and sends that data to a model trained to recognize handwritten digits.

Check out our _<a href="https://ibm.github.io/watsonx-ai-python-sdk/samples.html" target="_blank" rel="noopener no referrer">Online Documentation</a>_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### <a id="authors"></a>Authors

**Rafał Chrzanowski**, Software Engineer Intern at watsonx.ai.

Copyright © 2025 IBM. This notebook and its source code are released under the terms of the MIT License.